<a href="https://colab.research.google.com/github/shmilyface/DS-Unit-2-Tree-Ensembles/blob/master/Tanzania_Take_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
#installs
pip install category_encoders
pip install matplotlib==3.1.0

In [0]:
#imports
%matplotlib inline
import category_encoders as ce 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns 
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier

In [13]:
#functions
def wrangler(x):
  x = x.copy()
  x['date_recorded'] = pd.to_datetime(x['date_recorded'], infer_datetime_format=True)
  x['year_recorded'] = x['date_recorded'].dt.year 
  x['month_recorded'] = x['date_recorded'].dt.month 
  x['day_recorded'] = x['date_recorded'].dt.day
  x = x.drop(columns = 'date_recorded')
  x['years'] = x['year_recorded'] - x['construction_year']
  unusable_variance = ['recorded_by', 'id']
  x = x.drop(columns=unusable_variance)
  duplicate_columns = ['quantity_group']
  x=x.drop(columns=duplicate_columns)
  x['latitude'] = x['latitude'].replace(-2e-08, np.nan)
  
  cols_w_zeros = ['construction_year', 'longitude', 'latitude', 'gps_height', 'population']
  for col in cols_w_zeros:
    x[col] = x[col].replace(0, np.nan)

  return x

df_data = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Tree-Ensembles/master/data/tanzania/'

train = pd.merge(pd.read_csv(df_data + 'train_features.csv'),
                   pd.read_csv(df_data + 'train_labels.csv'))
  
test = pd.read_csv(df_data + 'test_features.csv')
sample_submission = pd.read_csv(df_data + 'sample_submission.csv')

train, val = train_test_split(train, test_size=len(test),
                              stratify=train['status_group'], random_state=42)

train = wrangler(train)
val = wrangler(val)
test = wrangler(test)

target = 'status_group'
x_train = train.drop(columns=target)
y_train = train[target]
x_val = val.drop(columns=target)
y_val = val[target]
x_test = test

pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(strategy='mean'),
    RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
)

pipeline.fit(x_train, y_train)
y_pred = pipeline.predict(x_val)
print('Validation Accuracy', accuracy_score(y_val, y_pred))


Validation Accuracy 0.8140409527789386


In [0]:
#submission
submission = sample_submission.copy()
submission['status_group'] = y_pred
submission.to_csv('submission-02.csv', index=False)